In [67]:
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
df = pd.read_csv("/Users/kjwdamme/PythonProjects/datasets/iot23_combined.csv")

In [3]:
df

,Unnamed: 0,id.orig_h,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,192.168.1.200,-1.0,-1,-1,0.0,136.0,8578.0,134.0,193266.0,...,0,0,0,0,0,0,0,1,0,0
1,1,192.168.1.200,-1.0,-1,-1,0.0,170.0,11602.0,170.0,244256.0,...,0,0,0,0,0,0,0,1,0,0
2,2,192.168.1.200,-1.0,-1,-1,0.0,6.0,360.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,3,192.168.1.200,-1.0,-1,-1,0.0,6.0,360.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,4,192.168.1.200,-1.0,-1,-1,0.0,6.0,360.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,99994,192.168.100.111,-1.0,-1,-1,0.0,2.0,80.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446617,99995,192.168.100.111,-1.0,-1,-1,0.0,2.0,80.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446618,99996,192.168.100.111,-1.0,-1,-1,0.0,2.0,80.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446619,99997,192.168.100.111,-1.0,-1,-1,0.0,2.0,80.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [4]:
df['label'].value_counts()

PartOfAHorizontalPortScan     825939
Okiru                         262690
Benign                        199756
DDoS                          138777
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: label, dtype: int64

In [5]:
df['id.orig_h'].value_counts()

192.168.1.197      204393
192.168.1.198      199987
192.168.100.111    199935
192.168.1.195      123106
192.168.100.103    100647
                    ...  
93.39.237.194           1
182.72.26.5             1
93.61.72.129            1
83.246.194.199          1
163.44.74.70            1
Name: id.orig_h, Length: 3296, dtype: int64

In [6]:
df_node = df[df['id.orig_h']=='192.168.1.198']

In [7]:
df_node.shape

(199987, 27)

In [8]:
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
Benign                        30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [9]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [10]:
df_node = df_node.drop(columns=['Unnamed: 0', 'id.orig_h'])

In [11]:
df_node

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
946626,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,C&C,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
946627,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,C&C,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
946628,0.761313,157,54305,4344.0,41.0,2381.0,40.0,56393.0,C&C-FileDownload,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
946629,0.783553,90,54305,10136.0,41.0,2302.0,40.0,56393.0,C&C-FileDownload,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
946630,3.138207,0,0,0.0,3.0,180.0,0.0,0.0,C&C,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346617,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1346618,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1346619,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1346620,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [12]:
x = df_node.loc[:, df_node.columns != 'label'].to_numpy()
df_y = df_node.loc[:, df_node.columns == 'label']
df_y = pd.get_dummies(df_y, columns=['label'])
y = df_y.to_numpy()

In [26]:
scaler = MinMaxScaler()

x_scaled = scaler.fit_transform(x)

In [27]:
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
Benign                        30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [117]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=42)

In [118]:
model = Sequential()
model.add(Dense(12, input_dim=x.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
gru_model = Sequential()
# gru_model.add(Dense(x.shape[1], activation='relu'))
gru_model.add(GRU(units=128, input_shape=x_train.shape))
gru_model.add(GRU(units=128))
gru_model.add(GRU(units=128))
gru_model.add(Dense(y.shape[1], activation='sigmoid'))

gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer gru_45: expected ndim=3, found ndim=2

In [119]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
 77030/159989 [=============>................] - ETA: 8s - loss: 0.9820 - acc: 0.5634

KeyboardInterrupt: 

In [103]:
preds = model.predict_classes(x_test)
num_correct = 0

for i in range(0, len(preds)):
    if preds[i] == np.argmax(y_test[i]):
        num_correct+=1

num_correct/len(preds)

0.15055752787639382

In [64]:
model.save("/Users/kjwdamme/PythonProjects/datasets/first_model_scaled.h5")

In [43]:
preds.shape

(39998,)

In [46]:
np.unique(preds, return_counts=True)

(array([0, 2, 3, 4]), array([    2,     2,    17, 39977]))

In [51]:
temp = np.transpose(y_test)


(6, 39998)

In [54]:
temp_df = pd.DataFrame(y_test)

In [62]:
temp_df[4].value_counts()

1    22662
0    17336
Name: 4, dtype: int64